In [ ]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F 

In [ ]:
## Channel Attention (CA) Layer
class CALayer(nn.Module):
    # reduction；降维比例为r=16
    def __init__(self, channel, reduction=16):
        super(CALayer, self).__init__()
        # global average pooling: feature --> point
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        # feature channel downscale a undpscale --> channel weight
        self.conv_du = nn.Sequential(
            # channel // reduction，输出降维，即论文中的1x1xC/r
            nn.Conv2d(channel, channel // reduction, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            # channel，输出升维，即论文中的1x1xC
            nn.Conv2d(channel // reduction, channel, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        '''
        x就是 HxWxC 通道  y是权重
        y权重通过上面方式求出，然后 和x求乘积
        使得重要通道权重更大，不重要通道权重减小
        '''
        y = self.avg_pool(x)
        y = self.conv_du(y)
        return x * y
